# 第１ステージ　ステップ1-10

In [1]:
import numpy as np

## ステップ１

### 1.1変数とは
- 省略

### 1.2 Variableクラスの実装

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data

In [3]:
data = np.array(1.0)
x = Variable(data)
x.data

array(1.)

In [4]:
x.data = np.array(2.)
x.data

array(2.)

### 1.3 【補足】Nunpyの多次元配列
- 省略

## ステップ２

### 2.2 Functionクラスの実装

In [6]:
class Function:
    def __call__(self, input):
        x = input.data # データを取り出す
        y = x ** 2 # 実際の計算
        output = Variable(y) # Variableとして返す
        return output

In [7]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [8]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


### 2.3 Functionクラスを使う

1. Functionクラスは基底クラスとして、全ての関数に共通する機能を実装する
2. 具体的な関数は、Functionクラスを継承したクラスで実装する

以上の点を考慮し、Functionクラスを実装

In [9]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 具体的な計算はforwardメソッドで行う。
        output = Variable(y)
        return output
    
    def forward(self, x):
        # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()

- Functionクラスを継承して、Squareクラスを実装してみる

In [10]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [12]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## ステップ３　関数の連結

### 3.1 Exp関数の実装

- Functionクラスを継承して、$y=e^x$を実装

In [13]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

### 3.2関数を連結する
- Functionクラスを継承したクラスを連結して、$y=(e^{x^2})^2$を実装
<br>
- 実装の流れ<br>
$x$ &rArr; Square() &rArr; $x^2$ &rArr; Exp() &rArr; $e^{x^2}$ &rArr; Square() &rArr; $(e^{x^2})^2$

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x) # x^2
b = B(a) # e^(x^2)
y = C(b) # (e^(x^2))^2
print(y.data)

1.648721270700128


## ステップ５　バックプロパゲーションの理論
- チェインルール
3.2の場合で$\frac{dy}{dx}$を計算してみる<br>
- 微分の公式通りであれば以下のようになる。
$$lim_{h \to 0} \frac{f(x+h)-f(x)}{h}$$
- 公式通りに微分することの問題点
1. 誤差が含まれやすい
2. 計算コストが大きすぎる。ニューラルネットの場合更新するパラメータが膨大

そこでチェインルールによるバックプロパゲーションの出番
<br>

3.2では順伝播は以下のようになっている<br>
$x$ &rArr; *A* &rArr; $x^2$ &rArr; *B* &rArr; $e^{x^2}$ &rArr; *C* &rArr; $(e^{x^2})^2$

ここで、
$$\frac{dy}{dx}=((\frac{dy}{dy}\frac{dy}{db})\frac{db}{da}\frac{da}{dx})$$
であることから、順伝播と逆伝播は以下のような関係といえる。<br>
$x$ &rArr; *A* &rArr; $a$ &rArr; *B* &rArr; $b$ &rArr; *C* &rArr; $y$<br>
<br>
$\frac{dy}{dx}=((\frac{dy}{dy}\frac{dy}{db})\frac{db}{da})\frac{da}{dx}$ &lArr; $A'(x)$ &lArr; $\frac{dy}{da}=(\frac{dy}{dy}\frac{dy}{db})\frac{db}{da}$ &lArr; $B'(a)$ &lArr; $\frac{dy}{db}=\frac{dy}{dy}\frac{dy}{db}$ &lArr; $C'(b)$ &lArr; $\frac{dy}{dy}=1$


## ステップ６　手作業によるバックプロパゲーション

### 6.1 Variableクラスの追加実装
- Variableクラスを通常の値(data)に加えて、それに対応する微分した値(grad)を持つよう拡張

In [15]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

### 6.2 Functionクラスの追加実装
- Functionクラスに２つの機能を追加
1. 微分の計算を行う逆伝播の機能(backwardメソッド)
2. forwardメソッドを呼ぶ際に、入力されたVariableインスタンスを保持する機能

In [16]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 具体的な計算はforwardメソッドで行う。
        output = Variable(y)
        self.input = input # 入力された変数を覚える
        return output
    
    def forward(self, x):
        # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

### 6.3 SquareとExpクラスの追加実装

In [17]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [18]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

### 6.4 バックプロパゲーションの実装

In [20]:
# 順伝播の実装
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x) # x^2
b = B(a) # e^(x^2)
y = C(b) # (e^(x^2))^2

In [21]:
# 逆伝播の実装
y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [28]:
# 定義通りの微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

def numerical_diff(f, x, eps=1e-4):
    x1 = Variable(x.data + eps)
    y1 = f(x1)
    y0 = f(x)
    return (y1.data - y0.data) / eps

h = np.array(1e-5)
x = Variable(np.array(0.5))
x_dif = numerical_diff(f, x)
print(x_dif)

3.2981021178524195
